<a href="https://colab.research.google.com/github/lucaskpaulino/MaskR-CNN/blob/main/Mask_Detectron_LVIS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Instalação do Detectron 2 




---

São instaladas primeiramente suas dependências e em seguida o detectron


funçãão anti-afk colab

In [ ]:
# install dependencies: 
!pip install pyyaml==5.1
!pip install imantics
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version
# opencv is pre-installed on colab
# Helper function, used these for debugging purposes
# detector2 build only succeeds if CUDA version is correct

#!nvidia-smi
#!nvcc --version

#import torch
#torch.__version__
#import torchvision
#torchvision.__version__

!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

In [ ]:
#!pip install -q -U torch torchvision -f https://download.pytorch.org/whl/torch_stable.html 
#!pip install -q -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
#!pip install -q detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cpu/index.html
!pip install git+https://github.com/lvis-dataset/lvis-api.git
!pip install "git+https://github.com/philferriere/cocoapi.git#egg=pycocotools&subdirectory=PythonAPI"
!pip install git+https://github.com/philferriere/cocoapi.git#subdirectory=PythonAPI

**Importações necessárias para rodar o detectron2**

In [ ]:

# Base setup:
# detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# common libraries
import numpy as np
import os, json, cv2, random
import matplotlib.pyplot as plt
%matplotlib inline

# detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.structures import BoxMode

# **Comando para visualizar a placa de vídeo disponibilizada pelo Colab**

In [ ]:
!nvidia-smi

## Importar e resgistrar Dataset


---
Aqui será utilizado o Dataset Cityscapes




## Treinamento


---



In [ ]:
%ls

In [ ]:
from detectron2.engine import DefaultTrainer
from detectron2.evaluation import COCOEvaluator
from detectron2.engine.hooks import HookBase
from detectron2.evaluation import inference_context
from detectron2.utils.logger import log_every_n_seconds
from detectron2.data import DatasetMapper, build_detection_test_loader
import detectron2.utils.comm as comm
import torch
import time
import datetime

class MyTrainer(DefaultTrainer):
    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        if output_folder is None:
            output_folder = os.path.join(cfg.OUTPUT_DIR, "inference")
        return COCOEvaluator(dataset_name, cfg, True, output_folder)
                     

**Aqui é criado um hook (gancho), que é responsável pela critério de parada (Early stopping)**

In [ ]:
def build_hooks(self):
        hooks = super().build_hooks()
        hooks.insert(-1,LossEvalHook(
            cfg.TEST.EVAL_PERIOD,
            self.model,
            build_detection_test_loader(
                self.cfg,
                self.cfg.DATASETS.TEST[0],
                DatasetMapper(self.cfg,True)
            )
        ))
        return hooks

In [ ]:
class LossEvalHook(HookBase):
   
      def __init__(self, eval_period, model, data_loader):
        self._model = model
        self._period = eval_period
        self._data_loader = data_loader
    
      def _do_loss_eval(self):
        # Copying inference_on_dataset from evaluator.py
        total = len(self._data_loader)
        num_warmup = min(5, total - 1)
            
        start_time = time.perf_counter()
        total_compute_time = 0
        losses = []
        for idx, inputs in enumerate(self._data_loader):            
            if idx == num_warmup:
                start_time = time.perf_counter()
                total_compute_time = 0
            start_compute_time = time.perf_counter()
            if torch.cuda.is_available():
                torch.cuda.synchronize()
            total_compute_time += time.perf_counter() - start_compute_time
            iters_after_start = idx + 1 - num_warmup * int(idx >= num_warmup)
            seconds_per_img = total_compute_time / iters_after_start
            if idx >= num_warmup * 2 or seconds_per_img > 5:
                total_seconds_per_img = (time.perf_counter() - start_time) / iters_after_start
                eta = datetime.timedelta(seconds=int(total_seconds_per_img * (total - idx - 1)))
                log_every_n_seconds(
                    logging.INFO,
                    "Loss on Validation  done {}/{}. {:.4f} s / img. ETA={}".format(
                        idx + 1, total, seconds_per_img, str(eta)
                    ),
                    n=5,
                )
            loss_batch = self._get_loss(inputs)
            losses.append(loss_batch)
        mean_loss = np.mean(losses)
        self.trainer.storage.put_scalar('validation_loss', mean_loss)
        comm.synchronize()

        return losses
            
        def _get_loss(self, data):
        # How loss is calculated on train_loop 
          metrics_dict = self._model(data)
          metrics_dict = {
            k: v.detach().cpu().item() if isinstance(v, torch.Tensor) else float(v)
            for k, v in metrics_dict.items()
          }
          total_losses_reduced = sum(loss for loss in metrics_dict.values())
        return total_losses_reduced
        
        
        def after_step(self):
          next_iter = self.trainer.iter + 1
          is_final = next_iter == self.trainer.max_iter
          if is_final or (self._period > 0 and next_iter % self._period == 0):
              self._do_loss_eval()
          self.trainer.storage.put_scalars(timetest=12)

# **Definição dos parâmetros e treinamento**

In [ ]:
%mkdir datasets
%mkdir datasets/lvis

In [ ]:
import gdown

!gdown https://drive.google.com/uc?id=1wcf6wb_RpYUqyZgsEa2eIrEIDrohsBQ9
!gdown https://drive.google.com/uc?id=1JDlvKip1PlgSRbR8msWxMXfu93-TdFLa

!unzip "lvis_v1_train.json.zip" -d "datasets/lvis"
!unzip "lvis_v1_val.json.zip" -d "datasets/lvis"

In [ ]:
!unzip "lvis_v1_image_info_test_dev.zip" -d "datasets/lvis"

In [ ]:
%mkdir datasets/coco

In [ ]:
!ls

In [ ]:
!wget http://images.cocodataset.org/zips/train2017.zip
!unzip "train2017.zip" -d "/content/datasets/coco"

In [ ]:
!wget http://images.cocodataset.org/zips/val2017.zip
!unzip "val2017.zip" -d "/content/datasets/coco"

In [ ]:
!wget http://images.cocodataset.org/zips/test2017.zip
!unzip "test2017.zip" -d "/content/datasets/coco"

In [ ]:
#from detectron2.data.datasets import register_coco_instances

#register_coco_instances("lucas_val1", {}, "coco-1634746569.6402342.json", "/content/datasets/coco/val2017")


In [ ]:
#from detectron2.data.datasets import register_coco_instances

#register_coco_instances("lucas_val1", {}, "coco-1634746569.6402342.json", "/content/datasets/coco/val2017")

In [ ]:
from detectron2.engine import DefaultTrainer
#from lvis import LVIS

from tensorflow import data
#AUTOTUNE = data.experimental.AUTOTUNE

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("LVISv1-InstanceSegmentation/mask_rcnn_R_50_FPN_1x.yaml"))

cfg.DATASETS.TRAIN = ("lvis_v1_train",)
cfg.DATASETS.TEST = ("lvis_v1_val",)

cfg.SOLVER.REFERENCEWORLDSIZE = 1
cfg.DATALOADER.NUM_WORKERS = 2
#cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("LVISv1-InstanceSegmentation/mask_rcnn_R_50_FPN_1x.yaml")  # Let training initialize from model zoo

cfg.SOLVER.IMS_PER_BATCH = 4
cfg.SOLVER.BASE_LR = 0.00001  # learning rate

cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 1024 # default 512
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1203

cfg.SOLVER.WARMUP_ITERS = 50
cfg.SOLVER.MAX_ITER = 600 # 1500 ajustar para overfit ou mAP subindo

cfg.SOLVER.STEPS = (300,500) #1000, 1500
cfg.SOLVER.GAMMA = 0.005

cfg.TEST.EVAL_PERIOD = 400

cfg.SOLVER.CHECKPOINT_PERIOD = 200



os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = MyTrainer(cfg)
#trainer.resume_or_load(resume=True)
trainer.train()

In [ ]:
%ls

In [ ]:
# Look at training curves in tensorboard:
%load_ext tensorboard
%tensorboard --logdir output
#!kill 640

In [ ]:
def rle_decode(data):
    decode = ''
    count = ''
    for char in data:
        # If the character is numerical...
        if char.isdigit():
            # ...append it to our count
            count += char
        else:
            # Otherwise we've seen a non-numerical
            # character and need to expand it for
            # the decoding
            decode += char * int(count)
            count = ''
    return decode

In [ ]:
import json

with open('/content/PKLot/PKLot/instances_ufpr05_test.json') as json_file:
    json_data = json.load(json_file)
    #print(json_data['annotations'])
data = json.dumps(json_data)

for annotations in json_data['annotations']:
  print(annotations['image_id'],annotations['segmentation'])


In [ ]:
import json
import pycocotools
import numpy as np
from imantics import Polygons, Mask
import pycocotools.mask as mask_util
from detectron2 import structures



with open('/content/PKLot/PKLot/output/inference/coco_instances_results.json') as json_predict:
    json_p = json.load(json_predict)
    #print(json_data)
    #print(json.dumps(json_data[1], indent=4))

data_p = json.dumps(json_p)
data2_p = json.loads(data_p)

with open('/content/PKLot/PKLot/instances_ufpr05_test.json') as json_gt:
    json_gt = json.load(json_gt)
    #print(json_data['annotations'])

data_gt = json.dumps(json_gt)
data2_gt = json.loads(data_gt)


for counts, annotations in zip(data2_p, data2_gt['annotations']):
  
  #for annotations in data2_gt['annotations']:
   # print(annotations['image_id'],annotations['segmentation'])
  #counts['image_id'],
    seg_mask = ( counts['segmentation'])
    mask = mask_util.decode(seg_mask)[:, :]
  
    polygons = Mask(mask).polygons()
   # if data2_gt[counts] =! 1:
    #print(counts['image_id'], polygons.segmentation)  
    #print(annotations['image_id'])
    if counts['image_id'] == annotations['image_id']:
      bboxes_gt = polygons.segmentation
      bboxes_pred = annotations['segmentation']
      #polygon_area(bboxes_gt, bboxes_pred)
      #IOUs = structures.pairwise_iou(bboxes_gt, bboxes_pred)
      print(bboxes_gt)
      print(bboxes_pred)
    #print(mask)

In [ ]:
def polygon_area(x, y):
    # Using the shoelace formula
    # https://stackoverflow.com/questions/24467972/calculate-area-of-polygon-given-x-y-coordinates
    return 0.5 * np.abs(np.dot(x, np.roll(y, 1)) - np.dot(y, np.roll(x, 1)))

In [ ]:
import json
import pycocotools
import numpy as np
from imantics import Polygons, Mask
import pycocotools.mask as mask_util

with open('/content/PKLot/PKLot/output/inference/coco_instances_results.json') as json_file:
    json_data = json.load(json_file)
    #print(json_data)
    #print(json.dumps(json_data[1], indent=4))

data = json.dumps(json_data)

data2 = json.loads(data)

for counts in data2:
  #counts['image_id'],
  seg_mask = ( counts['segmentation'])

In [ ]:
def get_building_dicts(img_dir):
    """This function loads the JSON file created with the annotator and converts it to
    the detectron2 metadata specifications.
    """
    # load the JSON file
    json_file = os.path.join(img_dir, "/content/PKLot/PKLot/instances_pucpr_train.json")
    with open(json_file) as f:
        imgs_anns = json.load(f)

    dataset_dicts = []
    # loop through the entries in the JSON file
    for idx, v in enumerate(imgs_anns.values()):
        record = {}
        # add file_name, image_id, height and width information to the records
        filename = os.path.join(img_dir, v["filename"])
        height, width = cv2.imread(filename).shape[:2]

        record["file_name"] = filename
        record["image_id"] = idx
        record["height"] = height
        record["width"] = width

        annos = v["regions"]

        objs = []
        # one image can have multiple annotations, therefore this loop is needed
        for annotation in annos:
            # reformat the polygon information to fit the specifications
            anno = annotation["shape_attributes"]
            px = anno["all_points_x"]
            py = anno["all_points_y"]
            poly = [(x + 0.5, y + 0.5) for x, y in zip(px, py)]
            poly = [p for x in poly for p in x]

            region_attributes = annotation["region_attributes"]["class"]

            # specify the category_id to match with the class.

            if "building" in region_attributes:
                category_id = 1
            elif "window" in region_attributes:
                category_id = 0

            obj = {
                "bbox": [np.min(px), np.min(py), np.max(px), np.max(py)],
                "bbox_mode": BoxMode.XYXY_ABS,
                "segmentation": [poly],
                "category_id": category_id,
                "iscrowd": 0,
            }
            objs.append(obj)
        record["annotations"] = objs
        dataset_dicts.append(record)

    return dataset_dicts


    # the data has to be registered within detectron2, once for the train and once for
# the val data
#for d in ["train", "val"]:
 #   DatasetCatalog.register(
 #       "buildings_" + d,
 #       lambda d=d: get_building_dicts("/content/" + d),
 #   )

building_metadata = MetadataCatalog.get("buildings_train")

dataset_dicts = get_building_dicts("/content/train")

## Teste de validação

In [ ]:
!sudo pip install -U numpy==1.11.0.


In [ ]:
from detectron2.data import DatasetCatalog, MetadataCatalog, build_detection_test_loader
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
cfg = get_cfg()

cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "/content/output/model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.50
cfg.DATALOADER.NUM_WORKERS = 2
predictor = DefaultPredictor(cfg)
evaluator = COCOEvaluator("lvis_v1_train", cfg, False, output_dir="./output/")
val_loader = build_detection_test_loader(cfg, "lvis_v1_train")
inference_on_dataset(trainer.model, val_loader, evaluator)
#instances_to_coco_json(instances, img_id)

visualizar as inferências


In [ ]:
from detectron2.utils.visualizer import ColorMode
import random
import matplotlib.pyplot as plt

dataset_dicts = DatasetCatalog.get('PKLot_test2')
for d in random.sample(dataset_dicts, 5):    
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)
    v = Visualizer(im[:, :, ::-1], metadata=MetadataCatalog.get(dataset_dicts), scale=0.8)
    v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    plt.figure(figsize = (14, 10))
    plt.imshow(cv2.cvtColor(v.get_image()[:, :, ::-1], cv2.COLOR_BGR2RGB))
    plt.show()

In [ ]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog


In [ ]:
!ls

In [ ]:
!wget http://images.cocodataset.org/val2017/000000439715.jpg -q -O input.jpg
im = cv2.imread("/content/datasets/coco/val2017/000000000139.jpg")
cv2_imshow(im)

In [ ]:
cfg = get_cfg()
# add project-specific config (e.g., TensorMask) here if you're not running a model in detectron2's core library
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model
# Find a model from detectron2's model zoo. You can use the https://dl.fbaipublicfiles... url as well
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
predictor = DefaultPredictor(cfg)
outputs = predictor(im)

In [ ]:
# look at the outputs. See https://detectron2.readthedocs.io/tutorials/models.html#model-output-format for specification
print(outputs["instances"].pred_classes)
print(outputs["instances"].pred_boxes)

In [ ]:
#outputs["instances"].pred_boxes.tensor.cpu().numpy()
#outputs["instances"].pred_classes.cpu().numpy()
print(outputs["instances"].pred_boxes.tensor.cpu().numpy())


In [ ]:
# We can use `Visualizer` to draw the predictions on the image.
v = Visualizer(im[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
cv2_imshow(out.get_image()[:, :, ::-1])

In [ ]:
def convert_to_coco_json(dataset_name, output_file, allow_cached=True):
    """
    Converts dataset into COCO format and saves it to a json file.
    dataset_name must be registered in DatasetCatalog and in detectron2's standard format.

    Args:
        dataset_name:
            reference from the config file to the catalogs
            must be registered in DatasetCatalog and in detectron2's standard format
        output_file: path of json file that will be saved to
        allow_cached: if json file is already present then skip conversion
    """

    # TODO: The dataset or the conversion script *may* change,
    # a checksum would be useful for validating the cached data

    PathManager.mkdirs(os.path.dirname(output_file))
    with file_lock(output_file):
        if PathManager.exists(output_file) and allow_cached:
            logger.warning(
                f"Using previously cached COCO format annotations at '{output_file}'. "
                "You need to clear the cache file if your dataset has been modified."
            )
        else:
            logger.info(f"Converting annotations of dataset '{PKLot_test}' to COCO format ...)")
            coco_dict = convert_to_coco_dict(PKLot_test)

            logger.info(f"Caching COCO format annotations at '{output_file}' ...")
            tmp_file = output_file + ".tmp"
            with PathManager.open(tmp_file, "w") as f:
                json.dump(coco_dict, f)
            shutil.move(tmp_file, output_file)

In [ ]:
from detectron2.data.datasets.coco import convert_to_coco_json
from detectron2.data import MetadataCatalog, DatasetCatalog
def data_dict():
    return [{
        'file_name': 'image.png',
        'image_id': 'image',
        'validated': 'false',
        'height': 100,
        'width': 100,
        'annotations': [{
            'bbox': [70, 30, 100, 70],
            ## area should be 30*40=1200
            'bbox_mode': BoxMode.XYXY_ABS,
            'category_id': 1,
            'iscrowd': 0,
        }]
    }]

def to_json():
    #cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained
   # cfg.DATASETS.TEST = ("PKLot_test")
   # cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5   # set a custom testing threshold
   # predictor = DefaultPredictor(cfg)
   # test_metadata = MetadataCatalog.get("PKLot_test")
   # register_coco_instances("pk_json4", {}, "output/coco_instances_results.json", "output/")
    #DatasetCatalog.register('pk_json3', data_dict)
   # MetadataCatalog.get('PKLot_test').set(thing_classes=["car"])
    #convert_to_coco_json('test_metadata', output_file='./output/test.json', allow_cached=False)

    #DatasetCatalog.register('test2', data_dict)
    MetadataCatalog.get('PKLot_test').set(thing_classes=["car"])
    convert_to_coco_json('PKLot_test', output_file='./output/test4.json', allow_cached=False)

if __name__ == '__main__':
    to_json()

In [ ]:
from google.colab import files
files.download('output/test4.json')

In [ ]:
import cv2
import numpy as np
import torch

from detectron2.structures import Instances, pairwise_iou_rotated, RotatedBoxes
from detectron2.utils.visualizer import Visualizer

boxes1 = torch.tensor([[160.0, 153.0, 230.0,  23.0, -37.0]], device='cuda')
boxes2 = torch.tensor([[190.0, 127.0,  80.0,  21.0, -46.0]], device='cuda')
print(pairwise_iou_rotated(RotatedBoxes(boxes1), RotatedBoxes(boxes2)))

instances = Instances((300, 300))
instances.pred_boxes = torch.cat((boxes1, boxes2)).cpu()
instances.scores = torch.tensor([1.0, 1.0])
instances.pred_classes = torch.tensor([0, 0])

viz = Visualizer(np.full((300, 300, 3), 127, dtype=np.uint8), {'thing_classes': ['thing']})
image = viz.draw_instance_predictions(instances).get_image()[:, :, ::-1]
cv2.imwrite('out.png', image)

## Inferência e teste

In [ ]:
im = cv2.imread("/content/PKLot/PKLot/UFPR05/Sunny/2013-02-22/2013-02-22_08_15_02.jpg")
cv2_imshow(im)

In [ ]:
# We can use `Visualizer` to draw the predictions on the image.
v = Visualizer(im[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
cv2_imshow(out.get_image()[:, :, ::-1])
outputs["instances"].pred_boxes.tensor.cpu().numpy()
outputs["instances"].pred_classes.cpu().numpy()


In [ ]:
from detectron2.utils.visualizer import ColorMode
import random
import matplotlib.pyplot as plt


test_metadata = MetadataCatalog.get("PKLot_test2")
dataset_dicts = DatasetCatalog.get('PKLot_test2')
for d in random.sample(dataset_dicts, 5):    
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)
    v = Visualizer(im[:, :, ::-1], metadata=test_metadata, scale=0.8)
    v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    plt.figure(figsize = (14, 10))
    plt.imshow(cv2.cvtColor(v.get_image()[:, :, ::-1], cv2.COLOR_BGR2RGB))
    plt.show()

In [ ]:
from detectron2.utils.visualizer import ColorMode
import glob

for imageName in glob.glob('/content/test/*jpg'):
  im = cv2.imread(imageName)
  outputs = predictor(im)
  v = Visualizer(im[:, :, ::-1],
                metadata=test_metadata, 
                scale=0.8
                 )
  out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
  cv2_imshow(out.get_image()[:, :, ::-1])


In [ ]:
import time
times = []
for i in range(20):
    start_time = time.time()
    outputs = predictor(im)
    delta = time.time() - start_time
    times.append(delta)
mean_delta = np.array(times).mean()
fps = 1 / mean_delta
print("Average(sec):{:.2f},fps:{:.2f}".format(mean_delta, fps))

In [ ]:
#%cd facebook/detectron2/